In [11]:
import pandas as pd

# 这里放第二阶段生成的结果，召回只考虑召回的效果
llm_output = pd.read_json('../llm_generate/data/stage2/dev_stage2.json')
# llm_output = pd.read_parquet('../recall/datas/stage2_376/dev_stage2_cv388.parquet')
llm_output.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId,MisconceptionName,QuestionId_Answer,text,recallMisconceptionId
0,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),A,\( \frac{62}{10} \),329,Does not understand a percentage is out of 100,6_A,Thinks that a percentage can be converted to a...,1955 847 318 122 1914 292 2361 1822 1786 1270 ...
1,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),C,\( \frac{6}{2} \),847,Thinks they can convert a 2 digit percentage t...,6_C,Thinks they can convert a 2 digit percentage t...,847 122 318 1955 1786 292 1221 1822 2368 2361 ...
2,6,376,Convert two digit integer percentages to fract...,238,Converting between Fractions and Percentages,Convert this percentage to a fraction\n\( 62 \...,B,\( \frac{31}{50} \),D,None of these,329,Does not understand a percentage is out of 100,6_D,Believes percentages cannot be converted into ...,804 847 122 1955 2361 318 145 1822 1396 1786 1...
3,7,314,Divide decimals by 10,224,Multiplying and Dividing with Decimals,\( 43.2 \div 10= \),A,\( 4.32 \),B,\( 0.432 \),2123,Divided by 100 rather than 10,7_B,"When dividing a decimal by a multiple of 10, j...",2133 1983 544 447 2272 2200 1619 2002 619 2525...
4,7,314,Divide decimals by 10,224,Multiplying and Dividing with Decimals,\( 43.2 \div 10= \),A,\( 4.32 \),C,\( 33.2 \),2273,Subtracts instead of divides,7_C,"When dividing a decimal by a multiple of 10, j...",2133 1983 544 2272 447 2200 1619 619 2002 2525...


In [12]:
import os
os.makedirs('./tmp_data1', exist_ok=True)
tmp = []
for i in range(len(llm_output)):
    text = f"ConstructName:{llm_output.loc[i, 'ConstructName']}\nQuestionText:{llm_output.loc[i, 'QuestionText']}\nCorrectAnswerText:{llm_output.loc[i, 'CorrectAnswerText']}\nInCorrectAnswerText:{llm_output.loc[i, 'InCorrectAnswerText']}\nmisconception:{llm_output.loc[i, 'text']}"
    # text = f"ConstructName:{llm_output.loc[i, 'ConstructName']}\nQuestionText:{llm_output.loc[i, 'QuestionText']}\nCorrectAnswerText:{llm_output.loc[i, 'CorrectAnswerText']}\nInCorrectAnswerText:{llm_output.loc[i, 'InCorrectAnswerText']}\nIncorrectReason:{llm_output.loc[i, 'cot_inference']}"
    tmp.append(text)
    
llm_output['text'] = tmp
print(llm_output['text'].values[0])
llm_output.to_parquet("./tmp_data1/recall_data.parquet", index=False)

ConstructName:Convert two digit integer percentages to fractions
QuestionText:Convert this percentage to a fraction
\( 62 \% \)
CorrectAnswerText:\( \frac{31}{50} \)
InCorrectAnswerText:\( \frac{62}{10} \)
misconception:Thinks that a percentage can be converted to a fraction using a denominator of 10 


In [13]:
%%time
# --model_dir=./rerank/models/bge-small-emb_100_v2/checkpoint-2355 
!/home/chenning/miniconda3/envs/llm_v2/bin/python run_similarity_search_nv.py \
    --recall_num 2587 \
    --gpu 2 \
    --input_file ./tmp_data1/recall_data.parquet \
    --output_file ./tmp_data1/recall_data_submission_stage2.csv \
    --num_beam 1 \
    --lora_path ../recall_model_save/llm_recall/nv_round1_qlora_rerun_v1_64_128/epoch_15_model/adapter.bin \
    --use_lora \



/home/chenning/opensource_models/nvidia/NV-Embed-v2
Namespace(model_dir='/home/chenning/opensource_models/nvidia/NV-Embed-v2', lora_path='../recall_model_save/llm_recall/nv_round1_qlora_rerun_v1_64_128/epoch_15_model/adapter.bin', input_file='./tmp_data1/recall_data.parquet', output_file='./tmp_data1/recall_data_submission_stage2.csv', batch_size=2, gpu=2, max_length=512, seed=777, recall_num=2587, num_beam=1, spread_max_length=False, temp=1.03, use_lora=True)
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:05<00:00,  1.48s/it]
/home/chenning/code/Eedi/llm_recall/run_similarity_search_nv.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value f

In [14]:
import pandas as pd
pred_labels = list(pd.read_csv("./tmp_data1/recall_data_submission_stage2.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.split()]).values)
# pred_labels  = list(pd.read_csv("submission.csv")["MisconceptionId"].apply(lambda x: [int(y) for y in x.strip('[]').replace('\n', ' ').split()]))

true_labels = list(pd.read_parquet("./tmp_data1/recall_data.parquet")["MisconceptionId"].values)

def mean_average_precision(true_labels, predicted_labels_list, k=25):

    average_precisions = []
    tmp = []
    for true_label, predicted_labels in zip(true_labels, predicted_labels_list):
        if true_label in predicted_labels[:k]:
            idx = predicted_labels[:k].index(true_label)
            average_precisions.append(1/(idx + 1))
            tmp.append(1)
        else:
            average_precisions.append(0)
            tmp.append(0)
    print(sum(tmp) / len(tmp) if tmp else 0)

    return sum(average_precisions) / len(average_precisions) if average_precisions else 0
    
score = mean_average_precision(true_labels,pred_labels)
print(score)


0.8516666666666667
0.42432930270200414
